In [3]:
import numpy as np
from numpy import inf
%run Functions.ipynb
%run Algorithms.ipynb

In [4]:
def optSolver_OptiVisioners(problem,method,options):
    
    x_list = list()
    f_list =  list()
    c_list = list()
    
    # compute initial function/gradient/Hessian (at x0)
    x = problem.x0 #get x0
    f = problem.compute_f(x) #Compute f at x0
    g = problem.compute_g(x) ### Compute g at x0
    H = problem.compute_H(x) #Compute H at x0 
    H_inv = np.identity(len(x))
   
    norm_g = np.linalg.norm(g, ord=inf) #Compute the infinity norm for g
    norm_g0 = np.linalg.norm(g, ord=np.inf)
    
    # set initial iteration counter that will stop the while loop according to our patience level
    k = 0
    c = 0
    delta = 1

    Stop=True #Boolean Variable
    
    while  norm_g > options.term_tol * np.max([norm_g0, 1]) and k < options.max_iterations:
       
        #Method name to implement the desired algorithm.
        
        if method.name == 'GradientDescent':
            x_new,f_new,g_new,c = GDStep(x,f,g,problem,method,options)
        elif method.name == 'Newton':
            x_new,f_new,g_new,H_new,c = Newton(x,f,g,H,problem,method,options) ### Add code!             
            H=H_new
        elif method.name == 'BFGS': 
            x_new,f_new,g_new,H_new,c = BFGS(x,f,g,H_inv,problem,method,options)
            H_inv=H_new
        elif method.name == 'DFP': 
            x_new,f_new,g_new,H_new,c = DFP(x,f,g,H_inv,problem,method,options)
            H_inv=H_new
        elif method.name == 'TRNewtonCG':
            x_new,f_new,g_new,H_new,delta_new = TRNewtonCG(x,f,g,H,delta,problem,method,options) 
            H=H_new
        elif method.name == 'TRSR1CG':
            x, f, x_list, f_list, c_list = TRSR1CG(x,f,g,H_inv,delta,problem,method,options)
            break
        else:
            print('Warning: method is not implemented yet')
        
        #Update to our new function values 
        x = x_new; f = f_new; g = g_new; norm_g = np.linalg.norm(g,ord=np.inf)
        
        x_list.append(x); f_list.append(f);c_list.append(c)
        # increment iteration counter
        k += 1 
    
    return x, f